# 第二题-第一小题

我从python pandas_datareader api中下载了20150101-20241231的Fama-French 3 factors in US的数据。

In [7]:
import pandas_datareader.data as web
import pandas as pd

# Define the start and end dates
start_date = '2015-01-01'
end_date = '2024-12-31'

# Fetch the Fama-French 3 factors
# The data is returned as a dictionary of DataFrames.
# The key 0 typically contains the monthly factors.
try:
    ff_factors_data = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start=start_date, end=end_date)
    # The monthly factors are usually in the first DataFrame (index 0)
    # The values are in percentages, so divide by 100
    ff_factors_monthly = ff_factors_data[0] / 100

    # Display the first few rows of the downloaded data
    print("Fama-French 3 Factors (Monthly) from {} to {}:".format(start_date, end_date))
    print(ff_factors_monthly.head())

    # Display the last few rows to check the end date
    print("\nLast few rows:")
    print(ff_factors_monthly.tail())

except Exception as e:
    print(f"An error occurred: {e}")
    print("Please ensure you have an internet connection and the Fama-French data library is accessible.")
    print("The Fama-French data might not be available up to the very end of the requested end_date if it hasn't been published yet.")

Fama-French 3 Factors (Monthly) from 2015-01-01 to 2024-12-31:
         Mkt-RF     SMB     HML   RF
Date                                
2015-01 -0.0310 -0.0059 -0.0345  0.0
2015-02  0.0613  0.0061 -0.0179  0.0
2015-03 -0.0111  0.0305 -0.0038  0.0
2015-04  0.0059 -0.0299  0.0180  0.0
2015-05  0.0137  0.0095 -0.0111  0.0

Last few rows:
         Mkt-RF     SMB     HML      RF
Date                                   
2024-08  0.0160 -0.0348 -0.0110  0.0048
2024-09  0.0172 -0.0013 -0.0277  0.0040
2024-10 -0.0100 -0.0099  0.0086  0.0039
2024-11  0.0649  0.0446  0.0016  0.0040
2024-12 -0.0317 -0.0271 -0.0299  0.0037


<ipython-input-7-3155c1971a9f>:12: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_factors_data = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start=start_date, end=end_date)
<ipython-input-7-3155c1971a9f>:12: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_factors_data = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start=start_date, end=end_date)


# 第二题-第二小题

此处提供了端到端的执行代码，您需要自行上传“行业ETF与无风险利率历史数据_2015_至_2024.xlsx”文件，这个文件您可以在github找到，同时我在网络学堂一并提交了。

In [5]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import pandas_datareader.data as web
from datetime import datetime

# --- Configuration ---
ETF_TICKERS = ["XLV", "DBA", "XME", "XLI", "XLU", "XLRE", "XRT", "IYT", "PEJ", "XLF"]
# This is the Excel file expected to be generated by your Task 1 .ipynb script
EXCEL_FILE_PATH = "行业ETF与无风险利率历史数据_2015_至_2024.xlsx"
START_DATE = '2015-01-01'
END_DATE = '2024-12-31' # Data will be fetched up to the latest available within this range

# --- 1. Data Loading and Preparation ---

def load_etf_monthly_returns(excel_path, tickers, start_date, end_date):
    """
    Loads ETF prices from the specified Excel file with multi-level headers,
    calculates monthly returns, and filters by date.
    """
    all_monthly_returns = pd.DataFrame()

    xls = pd.ExcelFile(excel_path)

    for ticker in tickers:
        try:
            # The .ipynb script saves raw ETF data in sheets named by their tickers.
            # Adjust header to [0, 1] for two-level header. index_col=0 assumes 'Date' is the first column.
            df_daily = pd.read_excel(xls, sheet_name=ticker, index_col=0, header=[0, 1], parse_dates=True)

            # Select the appropriate price column based on the new header structure.
            # We expect a column tuple like ('Price', 'XLV') for the ticker XLV.
            price_column_tuple = ('Price', ticker)

            if price_column_tuple in df_daily.columns:
                price_series = df_daily[price_column_tuple]
            else:
                # Fallback: if ('Price', ticker) is not found, try to find a 'Close' column,
                # assuming it might be ('Close', ticker) or a simple 'Close' if headers are inconsistent.
                close_column_tuple = ('Close', ticker)
                if close_column_tuple in df_daily.columns:
                    print(f"Warning: Column {price_column_tuple} not found for {ticker}. Using {close_column_tuple}.")
                    price_series = df_daily[close_column_tuple]
                elif 'Close' in df_daily.columns: # Check for a single level 'Close' column as a last resort
                    print(f"Warning: Column {price_column_tuple} and {close_column_tuple} not found for {ticker}. Using single-level 'Close' column.")
                    price_series = df_daily['Close']
                    if isinstance(price_series, pd.DataFrame): # If 'Close' itself has sub-columns
                        if ticker in price_series.columns:
                            price_series = price_series[ticker]
                        else: # Take the first column under 'Close'
                            price_series = price_series.iloc[:,0]
                else:
                    print(f"Error: Could not find a suitable price column (e.g., {price_column_tuple} or {close_column_tuple}) for {ticker}. Skipping.")
                    continue

            # Ensure price_series is a Series
            if isinstance(price_series, pd.DataFrame):
                if len(price_series.columns) == 1:
                    price_series = price_series.iloc[:, 0]
                else:
                    print(f"Error: Ambiguous price data for {ticker}; expected a Series but got DataFrame with columns: {price_series.columns}. Skipping.")
                    continue

            # Resample to monthly, taking the last available price of the month
            # ffill to handle potential missing days at month end before taking last
            price_series_monthly = price_series.resample('M').ffill().resample('M').last()
            monthly_returns = price_series_monthly.pct_change().dropna()
            all_monthly_returns[ticker] = monthly_returns

        except Exception as e:
            print(f"Could not load or process data for ETF {ticker} from {excel_path}: {e}")

    # Filter by date range
    all_monthly_returns = all_monthly_returns[(all_monthly_returns.index >= pd.to_datetime(start_date)) &
                                              (all_monthly_returns.index <= pd.to_datetime(end_date))]
    # Ensure index is PeriodIndex for merging with Fama-French data
    if not isinstance(all_monthly_returns.index, pd.PeriodIndex) and not all_monthly_returns.empty:
        all_monthly_returns.index = all_monthly_returns.index.to_period('M')

    return all_monthly_returns

def fetch_fama_french_factors(start_date, end_date):
    """
    Fetches Fama-French 3 factors (Mkt-RF, SMB, HML) and Risk-Free rate (RF).
    """
    try:
        # Fetch Fama-French 3 Factors. Returns a dictionary of DataFrames.
        # Key 0 usually contains monthly data.
        ff_data_dict = web.DataReader('F-F_Research_Data_Factors', 'famafrench',
                                      start=start_date, end=end_date)
        ff_factors = ff_data_dict[0] / 100 # Values are in percentages

        # Ensure index is PeriodIndex with monthly frequency
        if not isinstance(ff_factors.index, pd.PeriodIndex):
             ff_factors.index = ff_factors.index.to_period('M')

        print("Fama-French 3 Factors and RF (Monthly) fetched successfully:")
        print(ff_factors.head())
        return ff_factors

    except Exception as e:
        print(f"Error fetching Fama-French factors: {e}")
        print("Please ensure you have an internet connection and pandas_datareader is up to date.")
        print("Fama-French data might not be available for the full requested end_date.")
        return None

def prepare_regression_data(etf_returns, ff_factors):
    """
    Merges ETF returns with Fama-French factors and calculates excess returns.
    """
    if ff_factors is None or etf_returns.empty:
        print("Fama-French factors or ETF returns are missing. Cannot prepare data.")
        return pd.DataFrame(), pd.DataFrame()

    # Ensure RF is present
    if 'RF' not in ff_factors.columns:
        print("Error: 'RF' column missing from Fama-French factors.")
        return pd.DataFrame(), pd.DataFrame()

    # Merge ETF returns with Fama-French factors
    data = pd.merge(etf_returns, ff_factors, left_index=True, right_index=True, how='inner')

    # Calculate excess returns for each ETF
    excess_returns_cols = []
    for ticker in ETF_TICKERS: # Use the configured ETF_TICKERS list
        if ticker in data.columns:
            excess_return_col_name = f'{ticker}_ExRet'
            data[excess_return_col_name] = data[ticker] - data['RF']
            excess_returns_cols.append(excess_return_col_name)

    # Prepare factors for regression (Mkt-RF, SMB, HML)
    factors = data[['Mkt-RF', 'SMB', 'HML']]

    # Prepare ETF excess returns
    etf_excess_returns = data[excess_returns_cols]

    if etf_excess_returns.empty:
        print("No ETF excess returns could be calculated. Check data loading and merging.")

    print(f"\nData prepared for regression. Number of observations: {len(data)}")
    print("Sample of prepared data (first 5 rows):")
    print(data.head())

    return etf_excess_returns, factors

# --- 2. First-Pass Regression (Time-Series) ---

def run_first_pass_regressions(etf_excess_returns, factors):
    """
    Runs time-series regressions for each ETF's excess return on Fama-French factors.
    R_it - RF_t = alpha_i + b_i(Mkt-RF)_t + s_i*SMB_t + h_i*HML_t + e_it
    """
    if etf_excess_returns.empty or factors.empty:
        print("Cannot run first-pass regressions: data is missing.")
        return None, None

    results_summary = []
    beta_estimates_index = [col.replace('_ExRet', '') for col in etf_excess_returns.columns]
    # Initialize beta_estimates with dtype=float to prevent object dtype issues
    beta_estimates = pd.DataFrame(index=beta_estimates_index, columns=['alpha', 'beta_MktRF', 'beta_SMB', 'beta_HML'], dtype=float)

    X = sm.add_constant(factors)

    for ticker_ex_ret_col in etf_excess_returns.columns:
        ticker_name = ticker_ex_ret_col.replace('_ExRet', '')
        Y = etf_excess_returns[ticker_ex_ret_col].dropna()

        common_index = X.index.intersection(Y.index)
        if common_index.empty:
            print(f"No common data points for {ticker_name} after NaN handling. Skipping.")
            continue

        Y_aligned = Y.loc[common_index]
        X_aligned = X.loc[common_index]

        if Y_aligned.empty or X_aligned.empty or len(Y_aligned) < X_aligned.shape[1]:
            print(f"Data alignment resulted in insufficient data for {ticker_name}. Skipping.")
            continue

        try:
            model = sm.OLS(Y_aligned, X_aligned).fit()

            alpha = model.params.iloc[0]
            beta_MktRF = model.params.iloc[1]
            beta_SMB = model.params.iloc[2]
            beta_HML = model.params.iloc[3]

            t_alpha = model.tvalues.iloc[0]
            t_MktRF = model.tvalues.iloc[1]
            t_SMB = model.tvalues.iloc[2]
            t_HML = model.tvalues.iloc[3]

            r_squared = model.rsquared
            adj_r_squared = model.rsquared_adj
            num_obs = model.nobs

            results_summary.append({
                'ETF': ticker_name,
                'Alpha': alpha, 't_Alpha': t_alpha,
                'Beta_Mkt-RF': beta_MktRF, 't_Mkt-RF': t_MktRF,
                'Beta_SMB': beta_SMB, 't_SMB': t_SMB,
                'Beta_HML': beta_HML, 't_HML': t_HML,
                'R-Squared': r_squared,
                'Adj. R-Squared': adj_r_squared,
                'N_obs': num_obs
            })

            beta_estimates.loc[ticker_name] = [alpha, beta_MktRF, beta_SMB, beta_HML]

        except Exception as e:
            print(f"Error running first-pass regression for {ticker_name}: {e}")
            # Ensure the row in beta_estimates for this ticker still exists with NaNs
            beta_estimates.loc[ticker_name] = [np.nan, np.nan, np.nan, np.nan]


    if not results_summary:
        print("No first-pass regressions were successful.")
        return None, None

    first_pass_df = pd.DataFrame(results_summary)

    return first_pass_df, beta_estimates.drop(columns=['alpha'])

# --- 3. Second-Pass Regression (Cross-Sectional) ---

def run_second_pass_regression(avg_etf_excess_returns, first_pass_betas):
    """
    Runs cross-sectional regression of average ETF excess returns on their estimated betas.
    Avg_ExRet_i = gamma_0 + gamma_MktRF*beta_i_MktRF + gamma_SMB*beta_i_SMB + gamma_HML*beta_i_HML + u_i
    """
    # avg_etf_excess_returns is a Series (mean excess returns for each ETF)
    # first_pass_betas is a DataFrame (beta_MktRF, beta_SMB, beta_HML for each ETF)

    if avg_etf_excess_returns.empty or first_pass_betas is None or first_pass_betas.empty:
        print("Cannot run second-pass regression: input data is missing or empty.")
        return None, np.nan, np.nan, 0

    # Prepare independent variables (betas) and add a constant for the intercept
    X_ind_vars = first_pass_betas.copy()
    X_reg = sm.add_constant(X_ind_vars, prepend=True) # Adds 'const' column at the beginning

    # Prepare dependent variable (average excess returns)
    Y_dep = avg_etf_excess_returns.copy()

    # Align X and Y based on their common index (ETF names)
    common_index = X_reg.index.intersection(Y_dep.index)

    if common_index.empty:
        print("No common ETFs between betas and average excess returns for second pass. Skipping.")
        return None, np.nan, np.nan, 0

    X_aligned = X_reg.loc[common_index]
    Y_aligned = Y_dep.loc[common_index]

    # Ensure all data is numeric, converting non-numeric to NaN
    X_aligned = X_aligned.apply(pd.to_numeric, errors='coerce')
    Y_aligned = pd.to_numeric(Y_aligned, errors='coerce') # Y_aligned is a Series

    # Combine X and Y to handle NaNs consistently before regression
    temp_y_name = '_Y_DEP_VAR_' # Temporary unique name for Y column
    combined_df = pd.concat([Y_aligned.rename(temp_y_name), X_aligned], axis=1)

    # Drop any rows that have NaNs in *any* column (either Y or any X)
    combined_df.dropna(inplace=True)

    if combined_df.empty or len(combined_df) < X_aligned.shape[1]:
        print(f"After NaN handling, not enough data for second-pass regression. Valid observations: {len(combined_df)}")
        print(f"Required at least {X_aligned.shape[1]} observations for {X_aligned.shape[1]-1} regressors + intercept.")
        return None, np.nan, np.nan, 0

    # Separate back into Y and X for OLS
    Y_final = combined_df[temp_y_name]
    X_final = combined_df.drop(columns=[temp_y_name])

    try:
        model = sm.OLS(Y_final, X_final).fit()

        # Ensure correct mapping of coefficients based on X_final.columns
        coeff_names = ['gamma_0 (Intercept)'] + [f'gamma_{col}' for col in first_pass_betas.columns]


        results = {
            'Coefficient': coeff_names, # Use dynamic names based on beta columns
            'Estimate': model.params.values,
            'Std. Error': model.bse.values,
            't-statistic': model.tvalues.values,
            'p-value': model.pvalues.values
        }
        second_pass_df = pd.DataFrame(results)

        r_squared = model.rsquared
        adj_r_squared = model.rsquared_adj
        num_obs = model.nobs

        print("\n--- Second-Pass Regression Results ---")
        print(second_pass_df.to_string(index=False))
        print(f"\nR-Squared: {r_squared:.4f}")
        print(f"Adj. R-Squared: {adj_r_squared:.4f}")
        print(f"Number of observations (ETFs): {int(num_obs)}")
        print("\nNote: Standard errors in the second-pass regression are typically underestimated")
        print("due to the errors-in-variables problem (betas are estimated).")
        print("Consider Fama-MacBeth procedure or Shanken correction for more robust standard errors.")

        return second_pass_df, r_squared, adj_r_squared, int(num_obs)

    except Exception as e:
        print(f"Error running second-pass regression: {e}")
        return None, np.nan, np.nan, 0

# --- 4. Main Execution and Explanation ---

def main():
    print("--- Starting Fama-French Two-Pass Regression Analysis ---")

    # Step 1: Load and Prepare Data
    etf_monthly_returns = load_etf_monthly_returns(EXCEL_FILE_PATH, ETF_TICKERS, START_DATE, END_DATE)
    if etf_monthly_returns.empty:
        print("Failed to load ETF monthly returns. Exiting.")
        return

    ff_factors = fetch_fama_french_factors(START_DATE, END_DATE)
    if ff_factors is None:
        print("Failed to fetch Fama-French factors. Exiting.")
        return

    etf_excess_returns, factors_for_regression = prepare_regression_data(etf_monthly_returns, ff_factors)
    if etf_excess_returns.empty or factors_for_regression.empty:
        print("Failed to prepare regression data. Exiting.")
        return

    # Step 2: First-Pass Regressions
    print("\n--- Running First-Pass Time-Series Regressions ---")
    print("Model: R_it - RF_t = alpha_i + beta_i,Mkt(Mkt-RF)_t + beta_i,SMB*SMB_t + beta_i,HML*HML_t + e_it")

    first_pass_results_df, first_pass_betas = run_first_pass_regressions(etf_excess_returns, factors_for_regression)

    if first_pass_results_df is None or first_pass_betas is None:
        print("First-pass regressions failed or produced no betas. Exiting.")
        return

    print("\n--- First-Pass Regression Results Summary ---")
    cols_ordered = ['ETF', 'Alpha', 't_Alpha', 'Beta_Mkt-RF', 't_Mkt-RF',
                    'Beta_SMB', 't_SMB', 'Beta_HML', 't_HML',
                    'R-Squared', 'Adj. R-Squared', 'N_obs']
    print(first_pass_results_df[cols_ordered].to_string(index=False, float_format="%.4f"))

    # Step 3: Second-Pass Regression
    avg_etf_excess_returns = etf_excess_returns.mean()
    avg_etf_excess_returns.index = [col.replace('_ExRet', '') for col in avg_etf_excess_returns.index]

    print("\n--- Running Second-Pass Cross-Sectional Regression ---")
    print("Model: Avg_ExRet_i = gamma_0 + gamma_MktRF*beta_i,MktRF + gamma_SMB*beta_i,SMB + gamma_HML*beta_i,HML + u_i")

    if first_pass_betas.empty:
         print("First pass betas DataFrame is empty. Cannot run second pass. Exiting.")
         return

    common_idx = avg_etf_excess_returns.index.intersection(first_pass_betas.index)
    if common_idx.empty:
        print("No common ETFs between average excess returns and first pass betas. Exiting.")
        return

    avg_etf_excess_returns_aligned = avg_etf_excess_returns.loc[common_idx]
    first_pass_betas_aligned = first_pass_betas.loc[common_idx]

    second_pass_summary, _, _, _ = run_second_pass_regression(avg_etf_excess_returns_aligned, first_pass_betas_aligned)
    if second_pass_summary is None:
        print("Second-pass regression failed. Check logs for details.")

    # Step 4: Explanation of Results
    print("\n\n--- Explanation of Regression Results ---")

    print("\n**First-Pass Regressions (Time-Series):**")
    print("The first-pass regressions estimate the exposure of each ETF's excess return to the Fama-French factors (Mkt-RF, SMB, HML).")
    print("- **Alpha (Intercept):** Represents the average abnormal return of the ETF after accounting for its exposure to the Fama-French factors. A statistically significant alpha (t-statistic typically > |2|) suggests the ETF has outperformed or underperformed relative to what the model predicts. Ideally, alphas should be close to zero if the model fully explains returns.")
    print("- **Betas (Beta_Mkt-RF, Beta_SMB, Beta_HML):** Measure the sensitivity of the ETF's excess return to each factor.")
    print("  - Beta_Mkt-RF: Sensitivity to the market risk premium. A beta > 1 indicates higher sensitivity than the market; < 1 indicates lower.")
    print("  - Beta_SMB: Sensitivity to the size factor (small-cap premium). A positive beta indicates exposure to small-cap stocks.")
    print("  - Beta_HML: Sensitivity to the value factor (value stock premium). A positive beta indicates exposure to value stocks.")
    print("  - The t-statistics for these betas indicate whether the factor exposure is statistically significant.")
    print("- **R-Squared:** Indicates the proportion of the variance in an ETF's excess returns that is explained by the Fama-French factors. A higher R-squared suggests the model is a good fit for that particular ETF.")

    print("\n**Second-Pass Regression (Cross-Sectional):**")
    print("The second-pass regression tests whether the estimated factor exposures (betas) from the first pass can explain the cross-sectional differences in average ETF excess returns.")
    print("- **Gamma Coefficients (Estimates for gamma_Mkt-RF, gamma_SMB, gamma_HML):** These are the estimated risk premia associated with each factor. For example, gamma_Mkt-RF is the estimated market risk premium. If a factor is priced, its gamma should be statistically significant and have an economically plausible sign and magnitude (e.g., positive for market risk).")
    print("  - gamma_Mkt-RF: Expected premium for market risk.")
    print("  - gamma_SMB: Expected premium for size risk.")
    print("  - gamma_HML: Expected premium for value risk.")
    print("- **Gamma_0 (Intercept):** Represents the average excess return of a portfolio with zero exposure to all factors (zero betas). If the Fama-French model correctly prices assets, gamma_0 should be statistically indistinguishable from zero. A significant gamma_0 suggests model misspecification or unobserved risk factors.")
    print("- **t-statistics:** Indicate the statistical significance of the estimated gamma coefficients. As noted, these t-statistics might be biased due to the use of estimated regressors (betas).")
    print("- **R-Squared:** Indicates the proportion of the cross-sectional variation in average ETF excess returns that is explained by the differences in their factor betas. A higher R-squared suggests the Fama-French factors do a good job of explaining why some ETFs have higher average returns than others.")

    print("\n**Overall Interpretation:**")
    print("By examining both passes, you can assess how well the Fama-French 3-factor model explains the returns of your selected US industry ETFs. Significant alphas in the first pass or a significant gamma_0 in the second pass might indicate limitations of the model for this set of assets and time period.")
    print("The significance and signs of the factor risk premia (gammas) in the second pass indicate whether these factors commanded a premium in the market during the sample period.")

    print("\n--- Analysis Complete ---")

if __name__ == '__main__':
    main()


--- Starting Fama-French Two-Pass Regression Analysis ---


<ipython-input-5-4855d712954b>:66: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  price_series_monthly = price_series.resample('M').ffill().resample('M').last()
<ipython-input-5-4855d712954b>:66: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  price_series_monthly = price_series.resample('M').ffill().resample('M').last()


<ipython-input-5-4855d712954b>:66: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  price_series_monthly = price_series.resample('M').ffill().resample('M').last()


<ipython-input-5-4855d712954b>:66: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  price_series_monthly = price_series.resample('M').ffill().resample('M').last()


<ipython-input-5-4855d712954b>:66: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  price_series_monthly = price_series.resample('M').ffill().resample('M').last()
<ipython-input-5-4855d712954b>:66: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  price_series_monthly = price_series.resample('M').ffill().resample('M').last()


<ipython-input-5-4855d712954b>:66: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  price_series_monthly = price_series.resample('M').ffill().resample('M').last()
<ipython-input-5-4855d712954b>:66: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  price_series_monthly = price_series.resample('M').ffill().resample('M').last()


<ipython-input-5-4855d712954b>:66: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  price_series_monthly = price_series.resample('M').ffill().resample('M').last()
<ipython-input-5-4855d712954b>:66: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  price_series_monthly = price_series.resample('M').ffill().resample('M').last()


Fama-French 3 Factors and RF (Monthly) fetched successfully:
         Mkt-RF     SMB     HML   RF
Date                                
2015-01 -0.0310 -0.0059 -0.0345  0.0
2015-02  0.0613  0.0061 -0.0179  0.0
2015-03 -0.0111  0.0305 -0.0038  0.0
2015-04  0.0059 -0.0299  0.0180  0.0
2015-05  0.0137  0.0095 -0.0111  0.0

Data prepared for regression. Number of observations: 119
Sample of prepared data (first 5 rows):
              XLV       DBA       XME       XLI       XLU  XLRE       XRT  \
Date                                                                        
2015-02  0.042876 -0.003025  0.078171  0.053509 -0.063948   NaN  0.062433   
2015-03  0.006406 -0.040312 -0.067911 -0.025460 -0.009953   NaN  0.027238   
2015-04 -0.010896  0.006775  0.039721 -0.002510 -0.004726   NaN -0.042656   
2015-05  0.045043 -0.020637 -0.039618  0.003236  0.006332   NaN  0.010958   
2015-06 -0.003920  0.069629 -0.101149 -0.026547 -0.059840   NaN  0.011838   

              IYT       PEJ       XLF  ..

<ipython-input-5-4855d712954b>:89: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_data_dict = web.DataReader('F-F_Research_Data_Factors', 'famafrench',
<ipython-input-5-4855d712954b>:89: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_data_dict = web.DataReader('F-F_Research_Data_Factors', 'famafrench',
